In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import KFold, train_test_split

%matplotlib inline

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
train_df = pd.read_csv('https://grantmlong.com/data/SE_rents2018_train.csv', index_col=0)
test1_df = pd.read_csv('https://grantmlong.com/data/SE_rents2018_test1.csv', index_col=0)
test2_df = pd.read_csv('https://grantmlong.com/data/SE_rents2018_test2.csv', index_col=0)
test3_df = pd.read_csv('https://grantmlong.com/data/SE_rents2018_test3.csv', index_col=0)

#### Training Data

In [3]:
feature_cols = [
    'bedrooms', 'year_built', 'bathrooms', 'min_to_subway', 'size_sqft', 'no_fee', 'addr_zip', 'floornumber', 'floor_count', 
    'has_doorman', 'has_fireplace', 'has_gym', 'allows_pets', 'has_washer_dryer', 'has_garage',
    'has_roofdeck', 'has_concierge', 'has_pool', 'has_garden'

]

train_features = train_df[feature_cols] 

# impute missing values with medians
train_features = train_features.fillna(train_features.median(), axis=0)

# construct target vector
train_target = train_df['rent']

#### Handle Training Data Missing Values

In [4]:
def random_imputation(df, feature):

    number_missing = df[feature].isnull().sum()
    observed_values = df.loc[df[feature].notnull(), feature]
    df.loc[df[feature].isnull(), feature + '_imp'] = np.random.choice(observed_values, number_missing, replace = True)    
    return df

#missing_columns = ["year_built", "min_to_subway", "floornumber"]
#for feature in missing_columns:
#    test1_features[feature + '_imp'] = [feature]
#    test1_features = random_imputation(test1_features, feature)

test1_features = test1_df[feature_cols] 
test1_features = test1_features.fillna(train_features.median(), axis=0)

#  <font color=green> Test 1 <font>

#### Linear Regression Model

In [5]:
lreg = LinearRegression()
lreg.fit(train_features, train_target)

test1_df['predicted'] = lreg.predict(test1_features)
mean_squared_error(test1_df['rent'], test1_df['predicted'])
#test_df['predicted'].to_csv('linear.csv', header=True)
#3301338.418329301

3297096.0901556876

#### Random Forest Regression Model

In [6]:
regressor = RandomForestRegressor(n_estimators = 50, random_state = 1, max_depth=42)
regressor.fit(train_features, train_target)

test1_df['predicted'] = regressor.predict(test1_features)
mean_squared_error(test1_df['rent'], test1_df['predicted'])
#test_df['predicted'].to_csv('linear.csv', header=True)
#3301338.418329301

1746490.3228272318

#### Combine Data, Predict Values for Test 2

In [7]:
master_df = train_df.append(test1_df, sort=False)

master_features = master_df[feature_cols].fillna(master_df[feature_cols].median(), axis=0)
master_target = master_df['rent']

regressor.fit(master_features, master_target)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=42,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
           oob_score=False, random_state=1, verbose=0, warm_start=False)

#  <font color=green> Test 2 <font>


#### Handle Training Data Missing Values

In [8]:
test2_features = test1_df[feature_cols] 
test2_features = test1_features.fillna(train_features.median(), axis=0)

#### Random Forest Regression Model

In [9]:
test2_df['predicted'] = regressor.predict(test2_features)

In [10]:
test2_df['predicted'].to_csv('test2_results.csv', header=True)

#  <font color=green> Importing New Data <font>


In [11]:
dob_complaints_df = pd.read_csv('C:/Users/new/Downloads/data/DOB_Complaints_Received.csv', index_col=0)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (4,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [12]:
dob_violations_df = pd.read_csv('C:/Users/new/Downloads/data/DOB_Violations.csv', index_col=0)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [13]:
property_value_df = pd.read_csv('C:/Users/new/Downloads/data/Revised_Notice_of_Property_Value__RNOPV_.csv', index_col=0)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
#dob_complaints_df.sample(5)

In [15]:
#dob_violations_df.sample(5)

In [16]:
#property_value_df.sample(5).transpose()

In [17]:
property_value_df = property_value_df[['BIN','ORIGINAL MARKET VALUE']].copy()

In [18]:
#property_value_df.sample(5)

In [19]:
dob_violations_df['ISSUE_DATE'] =  pd.to_datetime(dob_violations_df['ISSUE_DATE'], format='%Y%m%d', errors='coerce')
dob_violations_df = dob_violations_df[(dob_violations_df['ISSUE_DATE'].dt.year > 2010) ]
bin_violations = dob_violations_df.BIN.value_counts().reset_index().rename(columns={'index': 'z', 0: 'count'})

In [20]:
master_df = train_df.append(test1_df, sort=False)

#master_df = pd.merge(left=master_df,right=bin_violations, left_on='bin', right_on='z')
master_df = pd.merge(left=master_df,right=bin_violations,  how='left', left_on='bin', right_on='z')
master_df.rename(columns={'BIN': 'dob_violations_count'}, inplace=True)
master_df.drop('z', axis=1, inplace=True)

In [21]:
#master_df = pd.merge(left=master_df,right=property_value_df, left_on='bin', right_on='ORIGINAL MARKET VALUE')
master_df = pd.merge(left=master_df,right=property_value_df,  how='left', left_on='bin', right_on='ORIGINAL MARKET VALUE')
master_df.rename(columns={'ORIGINAL MARKET VALUE': 'market_value'}, inplace=True)

master_df.drop('BIN', axis=1, inplace=True)

In [22]:
dob_complaints_df['Date Entered'] =  pd.to_datetime(dob_complaints_df['Date Entered'], format='%m/%d/%Y')
dob_complaints_df  = dob_complaints_df[(dob_complaints_df['Date Entered'].dt.year > 2010) ]
bin_complaints = dob_complaints_df.BIN.value_counts().reset_index().rename(columns={'index': 'z', 0: 'count'})


In [23]:
#master_df = pd.merge(left=master_df,right=bin_complaints, left_on='bin', right_on='z')
master_df = pd.merge(left=master_df,right=bin_complaints,  how='left', left_on='bin', right_on='z')
master_df.rename(columns={'BIN': 'dob_complaints_count'}, inplace=True)
master_df.drop('z', axis=1, inplace=True)


In [24]:
#master_df

In [ ]:
tax_df = pd.read_csv('C:/Users/new/Downloads/data/Property_Valuation_and_Assessment_data_Tax_Classes_2_3_4.csv', index_col=0)